In [30]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, log_loss, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [32]:
#importing files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('sample_submission.csv')

In [33]:
train.isna().sum()

user_id                0
REGION            157520
TENURE                 0
MONTANT           140277
FREQUENCE_RECH    140277
REVENUE           134663
ARPU_SEGMENT      134663
FREQUENCE         134663
DATA_VOLUME       196854
ON_NET            145819
ORANGE            166317
TIGO              239386
ZONE1             368310
ZONE2             374487
MRG                    0
REGULARITY             0
TOP_PACK          167329
FREQ_TOP_PACK     167329
CHURN                  0
dtype: int64

In [34]:
train.CHURN.value_counts()

0    325156
1     74844
Name: CHURN, dtype: int64

In [35]:
train['CHURN'].value_counts()

0    325156
1     74844
Name: CHURN, dtype: int64

In [36]:
train.describe()

,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN
count,259723.000000,259723.000000,265337.000000,265337.000000,265337.000000,203146.000000,254181.000000,233683.000000,160614.000000,31690.000000,25513.000000,400000.000000,232671.000000,400000.00000
mean,5522.971346,11.503733,5505.487757,1835.167658,13.951835,3369.763441,275.917586,95.532927,23.134608,7.874282,7.187003,28.046502,9.254209,0.18711
std,7099.640630,13.275514,7175.802367,2391.929290,14.679943,12281.867504,873.593034,204.733272,63.061871,35.642843,26.964028,22.282773,12.305563,0.39000
min,20.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000
25%,1000.000000,2.000000,1000.000000,333.000000,3.000000,0.000000,5.000000,7.000000,2.000000,0.000000,0.000000,6.000000,2.000000,0.00000
50%,3000.000000,6.000000,3000.000000,1000.000000,9.000000,267.000000,27.000000,29.000000,6.000000,1.000000,2.000000,24.000000,5.000000,0.00000
75%,7300.000000,15.000000,7340.000000,2447.000000,19.000000,2925.000000,155.000000,99.000000,20.000000,3.000000,5.000000,51.000000,12.000000,0.00000
max,226550.000000,133.000000,233413.000000,77804.000000,91.000000,934576.000000,45011.000000,6788.000000,2758.000000,1657.000000,1011.000000,62.000000,629.000000,1.00000


In [10]:
train.dtypes

user_id            object
REGION             object
TENURE             object
MONTANT           float64
FREQUENCE_RECH    float64
REVENUE           float64
ARPU_SEGMENT      float64
FREQUENCE         float64
DATA_VOLUME       float64
ON_NET            float64
ORANGE            float64
TIGO              float64
ZONE1             float64
ZONE2             float64
MRG                object
REGULARITY          int64
TOP_PACK           object
FREQ_TOP_PACK     float64
CHURN               int64
dtype: object

In [65]:
train['REGULARITY'].mean()

28.0465025

In [3]:
churn = train['CHURN']
labels = np.array(churn)
train = train.drop(['ZONE1', 'ZONE2', 'TIGO', 'user_id', 'CHURN'], axis=1)
test = test.drop(['ZONE1', 'ZONE2', 'TIGO', 'user_id'], axis=1)

In [4]:
def fill_missing_values(data):
    
    '''
    Function to input missing values based on the column object type
    '''
    
    cols = list(data.columns)
    for col in cols:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        
            data[col] = data[col].fillna(data[col].mean())
        
        #elif data[col].dtype == 'O' or data[col].dtype == 'object':
        #    data[col] = data[col].fillna(data[col].mode()[0])
            
        else:
            data[col] = data[col].fillna(data[col].mode()[0])
            
    return data

In [5]:
train1 = fill_missing_values(train)
test1 = fill_missing_values(test)

In [19]:
train1.dtypes

REGION             object
TENURE             object
MONTANT           float64
FREQUENCE_RECH    float64
REVENUE           float64
ARPU_SEGMENT      float64
FREQUENCE         float64
DATA_VOLUME       float64
ON_NET            float64
ORANGE            float64
MRG                object
REGULARITY          int64
TOP_PACK           object
FREQ_TOP_PACK     float64
dtype: object

In [21]:
train['REGION'].value_counts()

DAKAR          95517
THIES          33374
SAINT-LOUIS    22125
LOUGA          18513
KAOLACK        18042
DIOURBEL       12576
TAMBACOUNDA    10273
KAFFRINE        8181
KOLDA           7204
FATICK          6638
MATAM           5255
ZIGUINCHOR      4012
SEDHIOU          589
KEDOUGOU         181
Name: REGION, dtype: int64

In [6]:
enc_nom_1 = (train1.groupby('TOP_PACK').size()) / len(train1)
enc_nom_2 = (test1.groupby('TOP_PACK').size()) / len(test1)
enc_nom_1

TOP_PACK
1000=Unlimited7Day                  0.000010
200=Unlimited1Day                   0.000567
200=unlimited pilot auto            0.000003
200F=10mnOnNetValid1H               0.000053
301765007                           0.000003
                                      ...   
YMGX 100=1 hour FNF, 24H/1 month    0.000215
Yewouleen_PKG                       0.000192
pack_chinguitel_24h                 0.000003
pilot_offer5                        0.000003
pilot_offer6                        0.000003
Length: 111, dtype: float64

In [7]:
train1['TOP_PACK'] = train1['TOP_PACK'].apply(lambda x : enc_nom_1[x])
test1['TOP_PACK'] = test1['TOP_PACK'].apply(lambda x: enc_nom_2[x])

In [12]:
train1 = train1.drop('MRG', axis=1)
test1 = test1.drop('MRG', axis=1)

In [20]:
train1.dtypes

REGION             object
TENURE             object
MONTANT           float64
FREQUENCE_RECH    float64
REVENUE           float64
ARPU_SEGMENT      float64
FREQUENCE         float64
DATA_VOLUME       float64
ON_NET            float64
ORANGE            float64
REGULARITY          int64
TOP_PACK          float64
FREQ_TOP_PACK     float64
dtype: object

In [8]:
train1 = pd.get_dummies(train1, prefix=['REGION'], columns=['REGION'])
train1 = pd.get_dummies(train1, prefix=['TENURE'], columns=['TENURE'])

test1 = pd.get_dummies(test1, prefix=['REGION'], columns=['REGION'])
test1 = pd.get_dummies(test1, prefix=['TENURE'], columns=['TENURE'])

In [23]:
train1.shape, test1.shape

((400000, 33), (100000, 33))

In [14]:
## Randomly splitting into train and test data
nr.seed(9988)
indx = range(features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
x_train = features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [15]:
'''
    Randomly splitting the test data into two parts; one for validation during training
    and the other for testing the model
'''

np.random.seed(9988)
indx1 = range(x_test.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.5)
x_test1 = x_test[indx1[0],:]
y_test1 = np.ravel(y_test[indx1[0]])
x_test2 = x_test[indx1[1],:]
y_test2 = np.ravel(y_test[indx1[1]])

In [13]:
features = np.array(train1)
test_enc = np.array(test1)

scaler = preprocessing.StandardScaler().fit(features)
features = scaler.transform(features)
test_enc = scaler.transform(test_enc)

In [16]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_test1.shape, y_test1.shape)
print(x_test2.shape, y_test2.shape)

(280000, 33) (280000,)
(120000, 33) (120000,)
(60000, 33) (60000,)
(60000, 33) (60000,)


In [18]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [20]:
xg = xgb.XGBClassifier(boosting_type='gbtree', 
                      learning_rate=0.01, 
                      n_estimators=5000, 
                      max_depth=12, 
                      base_score=0.7,
                      max_features='auto'
                     )
eval_set = [(x_train, y_train), (x_test2, y_test2)]
xg.fit(x_train, y_train, eval_set=eval_set, eval_metric='logloss', early_stopping_rounds=200)

[11:17:31] WARNING: /workspace/src/learner.cc:480: 
Parameters: { boosting_type, max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-logloss:1.02962	validation_1-logloss:1.02936
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 200 rounds.
[1]	validation_0-logloss:1.01504	validation_1-logloss:1.01438
[2]	validation_0-logloss:1.00036	validation_1-logloss:0.99988
[3]	validation_0-logloss:0.98636	validation_1-logloss:0.98571
[4]	validation_0-logloss:0.97231	validation_1-logloss:0.97214
[5]	validation_0-logloss:0.95910	validation_1-logloss:0.95884
[6]	validation_0-logloss:0.94626	validation_1-logloss:0.94594
[7]	validation_0-logloss:0.93381	validation_1

[122]	validation_0-logloss:0.39189	validation_1-logloss:0.39730
[123]	validation_0-logloss:0.39027	validation_1-logloss:0.39571
[124]	validation_0-logloss:0.38870	validation_1-logloss:0.39415
[125]	validation_0-logloss:0.38711	validation_1-logloss:0.39261
[126]	validation_0-logloss:0.38546	validation_1-logloss:0.39110
[127]	validation_0-logloss:0.38392	validation_1-logloss:0.38961
[128]	validation_0-logloss:0.38242	validation_1-logloss:0.38812
[129]	validation_0-logloss:0.38091	validation_1-logloss:0.38668
[130]	validation_0-logloss:0.37932	validation_1-logloss:0.38526
[131]	validation_0-logloss:0.37784	validation_1-logloss:0.38384
[132]	validation_0-logloss:0.37641	validation_1-logloss:0.38241
[133]	validation_0-logloss:0.37499	validation_1-logloss:0.38106
[134]	validation_0-logloss:0.37362	validation_1-logloss:0.37968
[135]	validation_0-logloss:0.37217	validation_1-logloss:0.37836
[136]	validation_0-logloss:0.37081	validation_1-logloss:0.37701
[137]	validation_0-logloss:0.36934	valid

[251]	validation_0-logloss:0.28442	validation_1-logloss:0.29811
[252]	validation_0-logloss:0.28414	validation_1-logloss:0.29782
[253]	validation_0-logloss:0.28375	validation_1-logloss:0.29751
[254]	validation_0-logloss:0.28338	validation_1-logloss:0.29723
[255]	validation_0-logloss:0.28304	validation_1-logloss:0.29694
[256]	validation_0-logloss:0.28267	validation_1-logloss:0.29665
[257]	validation_0-logloss:0.28232	validation_1-logloss:0.29637
[258]	validation_0-logloss:0.28193	validation_1-logloss:0.29608
[259]	validation_0-logloss:0.28166	validation_1-logloss:0.29580
[260]	validation_0-logloss:0.28132	validation_1-logloss:0.29553
[261]	validation_0-logloss:0.28098	validation_1-logloss:0.29526
[262]	validation_0-logloss:0.28063	validation_1-logloss:0.29501
[263]	validation_0-logloss:0.28029	validation_1-logloss:0.29475
[264]	validation_0-logloss:0.27997	validation_1-logloss:0.29449
[265]	validation_0-logloss:0.27962	validation_1-logloss:0.29422
[266]	validation_0-logloss:0.27938	valid

[380]	validation_0-logloss:0.25689	validation_1-logloss:0.27866
[381]	validation_0-logloss:0.25679	validation_1-logloss:0.27859
[382]	validation_0-logloss:0.25662	validation_1-logloss:0.27855
[383]	validation_0-logloss:0.25651	validation_1-logloss:0.27849
[384]	validation_0-logloss:0.25641	validation_1-logloss:0.27842
[385]	validation_0-logloss:0.25632	validation_1-logloss:0.27837
[386]	validation_0-logloss:0.25622	validation_1-logloss:0.27831
[387]	validation_0-logloss:0.25611	validation_1-logloss:0.27825
[388]	validation_0-logloss:0.25594	validation_1-logloss:0.27820
[389]	validation_0-logloss:0.25585	validation_1-logloss:0.27814
[390]	validation_0-logloss:0.25576	validation_1-logloss:0.27810
[391]	validation_0-logloss:0.25567	validation_1-logloss:0.27804
[392]	validation_0-logloss:0.25557	validation_1-logloss:0.27800
[393]	validation_0-logloss:0.25546	validation_1-logloss:0.27794
[394]	validation_0-logloss:0.25537	validation_1-logloss:0.27789
[395]	validation_0-logloss:0.25521	valid

[509]	validation_0-logloss:0.24702	validation_1-logloss:0.27521
[510]	validation_0-logloss:0.24698	validation_1-logloss:0.27520
[511]	validation_0-logloss:0.24695	validation_1-logloss:0.27520
[512]	validation_0-logloss:0.24688	validation_1-logloss:0.27519
[513]	validation_0-logloss:0.24685	validation_1-logloss:0.27519
[514]	validation_0-logloss:0.24680	validation_1-logloss:0.27518
[515]	validation_0-logloss:0.24675	validation_1-logloss:0.27516
[516]	validation_0-logloss:0.24671	validation_1-logloss:0.27515
[517]	validation_0-logloss:0.24666	validation_1-logloss:0.27515
[518]	validation_0-logloss:0.24657	validation_1-logloss:0.27514
[519]	validation_0-logloss:0.24652	validation_1-logloss:0.27514
[520]	validation_0-logloss:0.24649	validation_1-logloss:0.27513
[521]	validation_0-logloss:0.24644	validation_1-logloss:0.27511
[522]	validation_0-logloss:0.24641	validation_1-logloss:0.27510
[523]	validation_0-logloss:0.24637	validation_1-logloss:0.27510
[524]	validation_0-logloss:0.24631	valid

[638]	validation_0-logloss:0.24297	validation_1-logloss:0.27477
[639]	validation_0-logloss:0.24295	validation_1-logloss:0.27477
[640]	validation_0-logloss:0.24293	validation_1-logloss:0.27477
[641]	validation_0-logloss:0.24291	validation_1-logloss:0.27476
[642]	validation_0-logloss:0.24288	validation_1-logloss:0.27476
[643]	validation_0-logloss:0.24285	validation_1-logloss:0.27477
[644]	validation_0-logloss:0.24283	validation_1-logloss:0.27477
[645]	validation_0-logloss:0.24281	validation_1-logloss:0.27477
[646]	validation_0-logloss:0.24279	validation_1-logloss:0.27477
[647]	validation_0-logloss:0.24278	validation_1-logloss:0.27477
[648]	validation_0-logloss:0.24276	validation_1-logloss:0.27477
[649]	validation_0-logloss:0.24274	validation_1-logloss:0.27477
[650]	validation_0-logloss:0.24271	validation_1-logloss:0.27477
[651]	validation_0-logloss:0.24270	validation_1-logloss:0.27477
[652]	validation_0-logloss:0.24268	validation_1-logloss:0.27477
[653]	validation_0-logloss:0.24266	valid

[767]	validation_0-logloss:0.24122	validation_1-logloss:0.27485
[768]	validation_0-logloss:0.24122	validation_1-logloss:0.27485
[769]	validation_0-logloss:0.24120	validation_1-logloss:0.27485
[770]	validation_0-logloss:0.24118	validation_1-logloss:0.27485
[771]	validation_0-logloss:0.24117	validation_1-logloss:0.27485
[772]	validation_0-logloss:0.24116	validation_1-logloss:0.27486
[773]	validation_0-logloss:0.24115	validation_1-logloss:0.27486
[774]	validation_0-logloss:0.24114	validation_1-logloss:0.27486
[775]	validation_0-logloss:0.24113	validation_1-logloss:0.27486
[776]	validation_0-logloss:0.24112	validation_1-logloss:0.27486
[777]	validation_0-logloss:0.24111	validation_1-logloss:0.27486
[778]	validation_0-logloss:0.24110	validation_1-logloss:0.27486
[779]	validation_0-logloss:0.24109	validation_1-logloss:0.27486
[780]	validation_0-logloss:0.24108	validation_1-logloss:0.27486
[781]	validation_0-logloss:0.24107	validation_1-logloss:0.27487
[782]	validation_0-logloss:0.24106	valid

XGBClassifier(base_score=0.7, booster='gbtree', boosting_type='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=12, max_features='auto', min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=5000,
              n_jobs=0, num_parallel_tree=1, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [1]:
#making prediction on the other part of the test data

pred = xg.predict_proba(x_test2)
pred

NameError: name 'xg' is not defined

In [23]:
confusion_matrix(y_test2, pred)

array([[45571,  3165],
       [ 4441,  6823]])

In [31]:
accuracy_score(pred, y_test2)

0.8732333333333333

In [24]:
#preparing the predictions make into the sample file for submission

preds1 = xg.predict(test_enc)
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = preds1
sample1.to_csv('XG_Baseline.csv', index = False)

In [29]:
sample1.CHURN.values[:50]

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1])

In [25]:
sample1.CHURN.value_counts()

0    83093
1    16907
Name: CHURN, dtype: int64

In [56]:
rf = RandomForestClassifier(n_estimators=1000, 
                            max_depth=12,
                            max_features="auto",
                            min_samples_split=2,
                            min_samples_leaf=1,
                            class_weight={0:0.8, 1:0.2})

rf.fit(x_train, y_train)

RandomForestClassifier(class_weight={0: 0.8, 1: 0.2}, max_depth=12,
                       n_estimators=1000)

In [57]:
#making prediction on the other part of the test data

pred = rf.predict(x_test1)
log_loss(pred, y_test1)

6.406515473753379

In [61]:
features.shape

(400000, 33)

In [68]:
'''
    After preparing your data from the above steps, model training can now happen.
    
    Using the sequential method imported from keras.model,
    a sequential Artificial Neural Network will be built
'''
#creating an instance of the sequential class
model = Sequential()

'''
    using model.add, we add layers to the model.
    
    Different types of layers exist, you find them all on the TensorFlow Keras documentations page
    The Dense layer is used for all layers in the model and it takes in a number of arguments.
    
    The first layer is the input layer
    and it is the only layer that takes the input_dim argument, the value should be the number of
    features/columns of data being passed into the model
    
    The model contains a total of four layers( 3 hidden layers and 1 output). The input layer is
    usually not counted as a layer
    
    args: The first parameter (number) refers to the number of neurons in that layer,
          it is also referred to as the number of outputs from that layer
          
    activation: Activation refers to the mathematical function/operation you want to 
                perform on the inputs coming into the layer before outputing it to the next layer
'''
#adding the layers to the models
model.add(Dense(20, input_dim=33, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='softmax'))
#model.add(Dense(1, kernel_initializer='normal'))

'''
    model.compile is used to 'compile' the model for training, by cobining all layers together,
    with the selected optimizer and loss metric to be used for training
    
    The Adam optimizer is used, other optimizer like SGD (Stochastic Gradient Descent) 
    occurs too but quite situable for specific tasks
'''

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [69]:
#Call backs are usually used to monitor training process, EarlySTopping is just one of these callbacks
#to stop model training after 40 epochs of which the validation has stopped improving

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=40)

valid_set = (x_test1, y_test1)   #validation set for training 

#fitting the neural network model
'''
model.fit args;
                epochs: number of times you want input to go through the neural net for training
                        Similar to number of estimators in tree based models
                batch_Size: refers to the number of input sample to go through the NN for each epoch
                callbacks: as defined above
'''
model.fit(x_train, y_train, batch_size=10, validation_data = valid_set, epochs=1500, callbacks=callback, shuffle=True, verbose=2)

Epoch 1/1500
28000/28000 - 229s - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/1500


KeyboardInterrupt: 